# Compustat World Firms Industry Classification

# 1. Data Cleaning and Feature Engineering

##  Import Libraries and Load Datasets

In [7]:
# Data analysis
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.stats import uniform, loguniform
from skopt.space import Real, Categorical, Integer
import pickle


# Preprocessing & Splitting
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GroupShuffleSplit, StratifiedGroupKFold

# Modeling
from skopt import BayesSearchCV 
import xgboost
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [8]:
comp_global = pd.read_csv('../data/compustat_global.csv')
comp_us = pd.read_csv('../data/compustat_north_america.csv')

## Initial Data Cleaning & Combine Datasets

### Align columns & combine datasets

In [9]:
# Drop columns w/ no predictive power (name) and unaligned columns.
comp_global.drop(['sic','ggroup','gind','gsubind','exchg',
                  'conm','conml','datadate','unnpq'], 
                 axis=1, inplace=True)
comp_us.drop(['indfmt','consol','popsrc','datafmt','tic','costat',
              'sic','ggroup','gind','gsubind','datacqtr','exchg',
                'conm','conml','datadate','xrdq','niq'], 
                 axis=1, inplace=True)

# Fiscal Period
comp_global['datafqtr'] = comp_global['fyearq'].astype(str)\
                                +'Q' + comp_global['fqtr'].astype(str)
comp_global = comp_global.sort_values('datafqtr')
sorted_unique_periods = sorted(comp_global['datafqtr'].unique())

# Fill Comp Global Currency with USD
comp_global['curcdq'] = None
comp_global['curcdq'].fillna('USD', inplace=True)

# Gross Profit = Sales - Cost of Goods Sold
comp_us['gpq'] = comp_us['saleq'] - comp_us['cogsq']

comp_global = comp_global[comp_us.columns]
compustat = pd.concat([comp_global, comp_us], ignore_index=True)

### Clean, order, filter dataset

In [10]:
# Sector
gic_to_sector = {
    0: 'Energy',
    1: 'Materials',
    2: 'Industrials',
    3: 'Consumer Discretionary',
    4: 'Consumer Staples',
    5: 'Health Care',
    6: 'Financials',
    7: 'Information Technology',
    8: 'Communication Services',
    9: 'Utilities',
    10: 'Real Estate'
}
compustat['gsector_num'] = compustat['gsector']/5-2
compustat['gsector'] = compustat['gsector_num'].map(gic_to_sector)

In [11]:
ordered_cols = ['gsector_num','gsector', # RESPONSE: industry sector
                'gvkey', # company ID
                'datafqtr','fyearq','fqtr', # response var and time
                'loc','curcdq', # categorical variables
                'saleq','gpq','oiadpq','oibdpq', # revenue / profit related variables
                'cogsq','xoprq', # cost related variables
                'atq','seqq','dlcq','dlttq','capxy','oancfy' # asset, debts, cashflow etc.
                ]
compustat = compustat[ordered_cols]
compustat = compustat.sort_values(['gvkey','datafqtr'])
compustat = compustat.reset_index(drop=True)

In [12]:
# Filter years >= 2000
compustat = compustat[(compustat["fyearq"]>=2000)]

# Remove companies where sector (y) is missing
mask = compustat.groupby('gvkey')['gsector'].transform(lambda x: x.isna().any())
compustat = compustat[~mask]

# Remove companies with no sales data
mask = compustat.groupby('gvkey')['saleq'].transform(lambda x: x.notna().any()) # remove if no sales data present
compustat = compustat[mask]
# Remove if any sales data is missing
#mask = compustat.groupby('gvkey')['saleq'].transform(lambda x: x.isna().any())
#compustat = compustat[~mask]

# Remove if any date information is missing
mask = compustat.groupby('gvkey')['datafqtr'].transform(lambda x: x.isna().any())
compustat = compustat[~mask]

In [13]:
compustat.shape

(2748178, 20)

In [14]:
compustat['gsector'].value_counts()

gsector
Industrials               508016
Consumer Discretionary    421159
Materials                 389118
Information Technology    385799
Health Care               262493
Consumer Staples          202318
Financials                167707
Energy                    147524
Communication Services    128289
Utilities                  87376
Real Estate                48379
Name: count, dtype: int64

In [15]:
compustat['gvkey'].nunique()

58930

In [7]:
#compustat.isna().mean().round(3) * 100

### Preliminary feature engineering (from domain knowledge)

In [8]:
# Gross Profit Margin = (Revenue - Cost_of_Goods_Sold) / Revenue
compustat['gpm'] = (compustat['saleq']-compustat['cogsq'])/compustat['saleq']

# Operating Profit Margin = Operating_Income (before depreciation) / Revenue
compustat['opm'] = (compustat['oibdpq'])/compustat['saleq']

# Operating Cash Flow Margin = Operating_Cash_Flow / Revenue
compustat['ocfm'] = (compustat['oancfy'])/compustat['saleq']

# Return on [Assets/Equity] = Operating_Income / Total_[Assets/Equity]
compustat['roa'] = (compustat['oiadpq'])/compustat['atq']
compustat['roe'] = (compustat['oiadpq'])/compustat['seqq']

# Cashflow to [Debt/Assets] Ratio = Operating_Cash_Flow / Total_[Debt/Assets]
compustat['cd_ratio'] = (compustat['oancfy'])/(compustat['dlcq']+compustat['dlttq'])
compustat['ca_ratio'] = (compustat['oancfy'])/compustat['atq']

# Free Cashflow to [Assets/Equity/Debt/Sales] Ratio = 
# (Operating_Cash_Flow - Capital_Expenditure) / Total_Assets
compustat['fca_ratio'] = (compustat['oancfy']-compustat['capxy'])/compustat['atq']
compustat['fce_ratio'] = (compustat['oancfy']-compustat['capxy'])/compustat['seqq']
compustat['fcd_ratio'] = (compustat['oancfy']-compustat['capxy'])/(compustat['dlcq']+compustat['dlttq'])
compustat['fcs_ratio'] = (compustat['oancfy']-compustat['capxy'])/compustat['saleq']

# Total Asset Turnover = Revenue / Total_Assets
compustat['tat'] = (compustat['saleq'])/compustat['atq']

# Current Ratio = Current_Assets / Current_Liabilities
compustat['cr'] = (compustat['atq']-compustat['dlcq']-compustat['dlttq'])/compustat['dlcq']

# Total Debt Ratio = Total_Debt / Total_Assets
compustat['tdr'] = (compustat['dlcq']+compustat['dlttq'])/compustat['atq']

# Debt to Equity Ratio = Total_Debt / Total_Equity
compustat['der'] = (compustat['dlcq']+compustat['dlttq'])/compustat['seqq']

ratios_to_cap = ['gpm','opm','ocfm','roa','roe','cd_ratio','ca_ratio',
                 'fca_ratio','fce_ratio','fcd_ratio','fcs_ratio','tat',
                 'cr','tdr','der']
for ratio in ratios_to_cap:
    compustat[ratio] = np.where(compustat[ratio].abs() > 2, 
                                np.sign(compustat[ratio]) * 2, compustat[ratio])

In [9]:
def create_lagged_features(df, columns, n_lags):
  lagged_features = []
  for col in columns:
    for lag in range(1, n_lags + 1):
      lagged_features.append(df.groupby('gvkey')[col].shift(lag).rename(f"{col}_lag{lag}"))
  return pd.concat(lagged_features, axis=1)

def create_sliding_window_features(df, columns, window_sizes):
  rolling_features = []
  for col in columns:
    for window_size in window_sizes:
      rolling_means = df.groupby('gvkey')[col].rolling(window=window_size).mean().reset_index(level=0, drop=True).rename(f"{col}_mean_{window_size}Q")
      rolling_stds = df.groupby('gvkey')[col].rolling(window=window_size).std().reset_index(level=0, drop=True).rename(f"{col}_std_{window_size}Q")
      rolling_features.append(pd.concat([rolling_means, rolling_stds], axis=1))
  return pd.concat(rolling_features, axis=1)


In [10]:
lag_slide_columns = ['gpm','opm','ocfm','roa','roe',
                     'fca_ratio','fce_ratio','fcd_ratio','fcs_ratio',
                     'tat','cr','tdr','der']
n_lags = 4
window_size1 = 4
window_size2 = 8

lagged_df = create_lagged_features(compustat, lag_slide_columns, n_lags)
rolling_df = create_sliding_window_features(compustat, lag_slide_columns, [window_size1, window_size2])

compustat = pd.concat([compustat, lagged_df, rolling_df], axis=1)

In [11]:
compustat.shape

(2748178, 139)

In [12]:
compustat.head()

,gsector_num,gsector,gvkey,datafqtr,fyearq,fqtr,loc,curcdq,saleq,gpq,...,cr_mean_8Q,cr_std_8Q,tdr_mean_4Q,tdr_std_4Q,tdr_mean_8Q,tdr_std_8Q,der_mean_4Q,der_std_4Q,der_mean_8Q,der_std_8Q
66,2.0,Industrials,1004,2000Q1,2000,1.0,USA,USD,241.770,39.109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,2.0,Industrials,1004,2000Q2,2000,2.0,USA,USD,211.335,39.853,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,2.0,Industrials,1004,2000Q3,2000,3.0,USA,USD,200.071,40.534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,2.0,Industrials,1004,2000Q4,2000,4.0,USA,USD,221.079,40.948,...,NaN,NaN,0.296097,0.013844,NaN,NaN,0.646195,0.051509,NaN,NaN
70,2.0,Industrials,1004,2001Q1,2001,1.0,USA,USD,202.993,34.164,...,NaN,NaN,0.303825,0.025066,NaN,NaN,0.665448,0.074448,NaN,NaN


In [13]:
a=compustat.isna().mean().round(3) * 100
a[a>0]

saleq           9.8
gpq             8.7
oiadpq         18.1
oibdpq         11.6
cogsq          11.5
               ... 
tdr_std_8Q     53.6
der_mean_4Q    44.3
der_std_4Q     44.3
der_mean_8Q    53.6
der_std_8Q     53.6
Length: 131, dtype: float64

In [14]:
#compustat.hist(figsize=(30,30))

In [15]:
compustat.shape

(2748178, 139)

In [16]:
#np.array(list(compustat.columns))

In [17]:
compustat.to_pickle('../data/compustat_ftreng.pkl')

In [18]:
#compustat = pd.read_pickle('../data/compustat_ftreng.pkl')